In [1]:
import os
import numpy as np
import pandas as pd
from keras.applications.densenet import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# Set the path to the dataset
data_dir = '/kaggle/input/animals-detection-images-dataset'

# Set the image size and batch size
image_size = (224, 224)
batch_size = 32

# Create an image data generator for the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Create a data generator for the training data
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

# Create an image data generator for the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# Create a data generator for the validation data
val_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

# Load the DenseNet121 model with pre-trained weights
base_model = DenseNet121(weights='imagenet', include_top=False)

# Add a global average pooling layer to the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a dense layer with the number of classes as the output size
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create a new model with the added layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model using the Adam optimizer and categorical crossentropy loss function
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up model checkpoint to save the best weights during training
checkpoint = ModelCheckpoint('animal_detection_model.h5', save_best_only=True)

# Train the model on the training data for 5 epochs, using the ModelCheckpoint callback
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[checkpoint])

Found 22566 images belonging to 80 classes.
Found 6505 images belonging to 80 classes.
29084464/29084464 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
706/706 [==============================] - 958s 1s/step - loss: 1.4365 - accuracy: 0.6396 - val_loss: 0.8591 - val_accuracy: 0.7500
Epoch 2/5
706/706 [==============================] - 791s 1s/step - loss: 0.7457 - accuracy: 0.7787 - val_loss: 0.7853 - val_accuracy: 0.7666
Epoch 3/5
706/706 [==============================] - 706s 1000ms/step - loss: 0.5828 - accuracy: 0.8121 - val_loss: 0.7488 - val_accuracy: 0.7742
Epoch 5/5
706/706 [==============================] - 690s 978ms/step - loss: 0.5455 - accuracy: 0.8200 - val_loss: 0.7735 - val_accuracy: 0.7783


In [1]:
from keras.models import load_model

# load the h5 model
model = load_model('./animal_detection_model.h5')

# Get a summary of the model's architecture
model.summary()

2023-04-18 22:10:59.063168: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-18 22:11:09.960805: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-18 22:11:09.979905: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 22:11:37.850610: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None, None,   0          ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1/conv (Conv2D)            (None, None, None,   9408        ['zero_padding2d[0][0]']         
                                64)                                                           

In [5]:
from keras.utils.vis_utils import plot_model

# Visualize model architecture
plot_model(model, to_file='model.png', show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [7]:
# Evaluate model performance on validation data
scores = model.evaluate(val_generator, verbose=1)
print('Validation loss:', scores[0])
print('Validation accuracy:', scores[1])


NameError: name 'val_generator' is not defined

In [8]:
import matplotlib.pyplot as plt

# Plot training loss and accuracy
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['accuracy'])
plt.title('Model Training')
plt.xlabel('Epoch')
plt.ylabel('Loss / Accuracy')
plt.legend(['Loss', 'Accuracy'])
plt.show()


AttributeError: 'NoneType' object has no attribute 'history'

In [1]:
import matplotlib.pyplot as plt
from keras.models import load_model

# Load the model from the h5 file
model = load_model('./animal_detection_model.h5')

# Get the training history
history = model.history

# Plot the accuracy and loss charts
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


2023-04-19 15:25:51.884900: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 15:26:03.473082: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 15:26:03.485675: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 15:26:12.646941: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


AttributeError: 'NoneType' object has no attribute 'history'

In [1]:
from tensorflow.keras.models import load_model

model = load_model('./animal_detection_model.h5')

2023-04-26 15:28:51.643672: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 15:28:51.736988: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 15:28:51.755778: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 15:28:53.792710: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class_names = model.class_names

AttributeError: 'Functional' object has no attribute 'class_names'

In [3]:
# Load the trained model
model = load_model('animal_detection_model.h5')

# Get the class indices
class_indices = model.class_indices

# Invert the dictionary to map the class names to their indices
class_indices_inv = {v: k for k, v in class_indices.items()}

# Get the list of class names
class_names = [class_indices_inv[i] for i in range(len(class_indices))]


AttributeError: 'Functional' object has no attribute 'class_indices'

In [4]:
from tensorflow.keras.models import load_model

model = load_model('./animal_detection_model.h5')

class_indices = train_generator.class_indices
animal_list = list(class_indices.keys())
print(animal_list)

NameError: name 'train_generator' is not defined